In [ ]:
%pip install pyspark findspark hadoop-azure azure-storage pymongo[srv] pandas delta-spark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession
from azure.storage.blob import BlobServiceClient
from delta import configure_spark_with_delta_pip
import pandas as pd
import os

# Configurações do Azure Data Lake
account_name = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
sas_token = os.environ["AZURE_STORAGE_SAS_TOKEN"]
account_url = f"https://{account_name}.blob.core.windows.net"

try:
    builder = SparkSession.builder.appName("Delta_Lake_with_SAS") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config(f"fs.azure.sas.bronze.{account_name}.blob.core.windows.net", sas_token)

    spark = configure_spark_with_delta_pip(builder).getOrCreate()
except Exception as e:
    print(f"Erro ao acessar configurar a sessão do spark: {e}")

# Listar todos os arquivos na camada Landing (com base em arquivos .parquet)
blob_service_client = BlobServiceClient(account_url=f"{account_url}?{sas_token}")
container_client = blob_service_client.get_container_client("landing-zone")

try:
    # Inicialize a lista de tabelas
    landing_tables = [blob.name for blob in container_client.list_blobs() if blob.name.endswith(".json")]
    print(f"Tabelas encontradas no container 'landing': {landing_tables}")
except Exception as e:
    print(f"Erro ao acessar o container ou listar blobs: {e}")

if not landing_tables:
    print("Nenhum arquivo .parquet encontrado no container.")
else:
    for landing_path in landing_tables:
        try:
            table_name = landing_path.split(".")[0]
            bronze_path = f"wasbs://bronze@{account_name}.blob.core.windows.net/{table_name + ".parquet"}"

            landing_blob_client = blob_service_client.get_blob_client(container="landing-zone", blob=landing_path)
            bronze_tblob_client = blob_service_client.get_blob_client(container="bronze", blob=landing_path)

            json_data = landing_blob_client.download_blob().readall()

            df = spark.createDataFrame(json_data)

            # Salvar os dados no formato Delta no Azure Blob Storage
            df.write.format("delta").mode("overwrite").save(bronze_path)

            print(f"Tabela {table_name} transformada e salva em {bronze_path}")
        except Exception as e:
            print(f"Erro ao processar a coleção {table_name}: {e}")


Erro ao acessar configurar a sessão do spark: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.
Tabelas encontradas no container 'landing': ['cars.json', 'clients.json', 'employees.json', 'payments.json', 'sales.json', 'services.json']
Erro ao processar a coleção cars: name 'spark' is not defined
Erro ao processar a coleção clients: name 'spark' is not defined
Erro ao processar a coleção employees: name 'spark' is not defined
Erro ao processar a coleção payments: name 'spark' is not defined
Erro ao processar a coleção sales: name 'spark' is not defined
Erro ao processar a coleção services: name 'spark' is not defined
